In [9]:
%pip install --user -q dotenv pre-commit

Note: you may need to restart the kernel to use updated packages.


In [31]:
DIRECTORY = "allset"  

PERIODS = ["1HRS"]  # 5초, 1분 단위 데이터

exchanges = [  # https://coinmarketcap.com/rankings/exchanges/
    "BINANCE", "COINBASE", "OKEX", "CRYPTOCOM", "BITGET",
    "KRAKEN", "KUCOIN", "MEXC", "GATEIO", "BYBIT", "HTX", "GATEIO",
    "BITFINEX", "HUOBI", "BITMART", "LBANK"
]  # "BYBIT"

In [32]:
import os
from coinapy.fetcher import get_exchange_symbols, get_ohlcv
from coinapy.util import find_concat_candidates, concat_csv
from coinapy.util import create_folders, kst_to_utc, look_symbols_in_exchange


os.makedirs(DIRECTORY, exist_ok=True)

print(kst_to_utc("2025-03-05T21:15:00.010000+09:00"))

# look_symbols_in_exchange("BINANCE")

2025-03-05T12:15:00.0100000Z


- Upbit Withdrawl Fee: https://upbit.com/service_center/guide 
    - http://upbit.com/service_center/notice?id=4446

In [33]:
concat_candidate = list(filter(
    lambda x: x,
    (
        find_concat_candidates(x, doSort=False)
        for x in (
            f"{y}/{symbol}"
            for y in (
                f"{DIRECTORY}/{period}/{exc}"
                for period in PERIODS
                for exc in exchanges
                if os.path.exists(f"{DIRECTORY}/{period}/{exc}")
            )
            for symbol in os.listdir(y)
        )
        if os.path.exists(x)
    )
))

for x in concat_candidate:
    concat_csv(*x)

In [53]:
currency_order = [
    'USD', 'USDT', 'USDC', 
    # 'KRW', 'EUR', 'CNY', 'JPY',
    # "FDUSD", "USD0", "PYUSD", "FRAX",
    # "DAI"
    ]
# GBP

# 시가총액 기준 상위 1천개 
# 89등
targets = [  # https://coinmarketcap.com/view/stablecoin/
    'DAI', 'USDE', "USDC", "USDT", "FDUSD", "USD0", "PYUSD", "FRAX",
    # "TUSD", "USDY", "RON", "AXS", "MANA", "HNT", "CRV", "ZEC", "WIF", "GRASS",
    # "DYDX", "CAKE", "STRK", "CFX", "AERO", "NFT", "XCN", "AR", "FTT",
    # "CORE", "KAVA", "CHZ", "XEC", "SPX", "COMP", "KAITO", "TWT", "MELANIA",
    # "RUNE", "APE", "PENDLE", "PENGU", "AXL", "MORPHO", "AMP", "AIOZ", "BRETT",
    # "GNO", "BNX", "RSR", "MINA", "LUNC", "LUNC", "AKT", "BEAM", "JST", "1INCH",
    # "DEEP", "ZK", "GLM", "EIGEN", "SNX", "DASH", "W", "SFP", "CTC", "TFUEL", "MX",
    # "BLUR", "SATS", "CKB", "ZIL", "FARTCOIN", "ROSE", "ASTR", "AI16Z", "ATH",
    # "SAFE", "QTUM", "BAT", "NOT", "ZKJ", "LPT", "CVX",  "ZRO", "ZRX", "VTHO", "ACH",
    # "MASK", "HOT", "WEMIX", "OSMO", "MOG", "POPCAT", "DCR", "GAS", "BabyDoge", "XCH",
    # "CELO", "ID", "ORDI", "MOCA", "SC", "ETHFI", "ANKR", "ONE", "DRIFT", "MEW", "SUSHI",
    # "ENJ", "XEM",  "AIC", "LUNA",
    "BTC", "ETH", "BNB", "ADA", "SOL", "DOT", "DOGE", "AVAX", "HBAR",
    "LEO", "SUI", "SHIB", "TON", "DOT", "OM", "BGB", "HYPE", "APT", "NEAR",
    "ONDO", "ICP", "PEPE", "OKB", "TRUMP", "VET", "MNT", "TAO", "INJ",
    # "STX", "IMX", "THETA", "SEI", "FLR", "WLD", "GRT", "QNT", "LDO", "BONK",
    # "NEXO", "SAND", "ENS", "BERA", "BTT", "IOTA", "XAUt", "JTO", "BSV", "KAIA",
    # "FLOW", "GALA", "FLOKI", "NEO", "PAXG", "PYTH", "EGLD", "RAY",
    "CRO", "POL", "KAS", "TIA", "RENDER", "GT", "ARB", "DEXE",
    "OP", "JUP", "S", "ENA", "FET", "KSC", "IP", "MOVE", "MKR", "XDC",
    "LINK", "XRP", "LTC", "BCH", "ALGO", "ATOM", "XTZ", "FIL", "TRX", "VET",
    "EOS", "XLM",  "AAVE", "NEO", "THETA", "XMR", "ETC",
    "MKR",
]

targets = list(set(targets))
print(len(targets), len(currency_order), len(targets) * len(currency_order))
print(len(targets) * len(currency_order) * (24 * (60//60) * 30) / 100)
# targets = [
#     "LINK",
# ]

69 3 207
1490.4


In [54]:
time_start = kst_to_utc("2024-01-15T00:00:00.010000+09:00")
time_end = kst_to_utc("2024-01-31T00:00:00.010000+09:00")
limit = 100000

In [55]:
%pwd

'c:\\Users\\32mou\\Documents\\coin_api_python'

In [56]:
from coinapy.util import aggregate_data
import time


fnames = []

for target in targets:
    symbols = get_exchange_symbols(
        target, exchanges,
        types=["spot"],
        currency=currency_order,
    )

    if not symbols:
        print(target, ": No Symbols")
        continue

    create_folders(DIRECTORY, PERIODS, symbols)
    print(symbols)

    for exchange, symbol_id in symbols.items():
        print(f"📌 {exchange} {target} symbol_id: {symbol_id}")
        for s in (s for s in symbol_id if s):
            for period in PERIODS:
                filename = os.path.join(
                    DIRECTORY, period, exchange, "_".join(s.split("_")[1:]), 
                    f'{time_start.replace(":", "_")}_{time_end.replace(":", "_")}.csv')
                if os.path.exists(filename):
                    print(f"📂 {s} already exists")
                    fnames.append(filename)
                    continue
                
                try:
                    data = get_ohlcv(s, period, time_start, time_end, limit=limit)
                except Exception as e:
                    print(f"⏰ {s} {e}")
                    time.sleep(1)
                    continue
                if data.empty:
                    time.sleep(1)
                    continue

                data.to_csv(filename, index=False)
                fnames.append(filename)

{'GATEIO': ['GATEIO_SPOT_LINK_USDT', 'GATEIO_SPOT_LINK_USD', 'GATEIO_SPOT_LINK_USDC'], 'OKEX': ['OKEX_SPOT_LINK_USDT', 'OKEX_SPOT_LINK_USDC', 'OKEX_SPOT_LINK_USD'], 'BINANCE': ['BINANCE_SPOT_LINK_USDT', 'BINANCE_SPOT_LINK_USDC'], 'COINBASE': ['COINBASE_SPOT_LINK_USD', 'COINBASE_SPOT_LINK_USDT'], 'KRAKEN': ['KRAKEN_SPOT_LINK_USD', 'KRAKEN_SPOT_LINK_USDT', 'KRAKEN_SPOT_LINK_USDC'], 'BITMART': ['BITMART_SPOT_LINK_USDT'], 'KUCOIN': ['KUCOIN_SPOT_LINK_USDT', 'KUCOIN_SPOT_LINK_USDC'], 'BITFINEX': ['BITFINEX_SPOT_LINK_USD', 'BITFINEX_SPOT_LINK_USDT'], 'CRYPTOCOM': ['CRYPTOCOM_SPOT_LINK_USDT', 'CRYPTOCOM_SPOT_LINK_USD', 'CRYPTOCOM_SPOT_LINK_USDC'], 'BITGET': ['BITGET_SPOT_LINK_USDT', 'BITGET_SPOT_LINK_USDC'], 'MEXC': ['MEXC_SPOT_LINK_USDT', 'MEXC_SPOT_LINK_USDC'], 'LBANK': ['LBANK_SPOT_LINK_USDC'], 'HUOBI': ['HUOBI_SPOT_LINK_USDT']}
📌 GATEIO LINK symbol_id: ['GATEIO_SPOT_LINK_USDT', 'GATEIO_SPOT_LINK_USD', 'GATEIO_SPOT_LINK_USDC']
📂 GATEIO_SPOT_LINK_USDT already exists
Empty Data
Empty Data
📌 

In [ ]:
os.makedirs("output", exist_ok=True)
output_fname = "output/allset-1mon-1MIN_stablecoin.csv"
df = aggregate_data(
    output_fname,
    PERIODS, exchanges,
    [
        f"{time_start}_{time_end}",
    ]
)

print(f"📊 {output_fname}")

📊 output/martial-law-5day-5SEC_stablecoin.csv


In [13]:
# aggregate_data(
#     PERIODS, exchanges, time_start + "_" + time_end
# )